In [37]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
if not anthropic_api_key:
    raise ValueError("ANTHROPIC_API_KEY is not set in the .env file.")

from langchain_anthropic import ChatAnthropic

In [38]:
ticker = 'MEDP'
year = 2024    
quarter = 4    

prompt_title = 'Summarize Financial Earnings Call Transcript in Structured Format'

prompt_body = '''Role/Context Setting
You are an experienced financial analyst with a deep understanding of corporate earnings calls. You will receive a raw transcript of a company’s quarterly earnings call. Your task is to produce a clear, concise summary of the call, following the specific structure and format below:

Highlight Summary

Provide a brief overview of the company’s most notable achievements and results for the quarter.
Include key figures (e.g., revenue, growth percentages, operating income) and any standout points.

Summary of CEO’s Comments

Summarize the CEO’s main points, focusing on strategic directions, significant announcements, and the tone of the CEO’s commentary.
Mention any major themes the CEO emphasized (e.g., innovation, market expansion, cost optimization).

Financial Performance

Delve deeper into the financial metrics (e.g., revenue breakdown by segment, operating margins, net income, EPS, cash flow).
Highlight year-over-year and quarter-over-quarter comparisons, if available.
Include reasons or factors driving the performance.

Business Highlights

Discuss any major product launches, partnerships, mergers/acquisitions, or market expansion efforts mentioned.
Emphasize notable operational successes or challenges.
Include relevant information on segments (e.g., cloud services, e-commerce, advertising) if applicable.

Future Outlook

Summarize forward-looking statements regarding strategy, guidance, and expectations for upcoming quarters or fiscal years.
Incorporate any commentary on macroeconomic trends, changes in consumer behavior, or new areas of investment.

Q&A

Capture the key questions posed by analysts or investors and the company’s responses.
Focus on areas that reveal new insights, concerns, or clarifications about the company’s strategy or performance.

Important Details to Include

Where possible, cite specific numbers, percentages, or guidance ranges.
Present information in clear bullet points or concise paragraphs.
Maintain a neutral and professional tone.
Avoid speculative or subjective language; base statements on what is clearly provided in the transcript.

Formatting

Use headings corresponding to the six sections (Highlight Summary, Summary of CEO’s Comments, Financial Performance, Business Highlights, Future Outlook, Q&A).
Write in a manner that is easily scannable (short paragraphs).
'''

In [39]:
# Folder where transcript JSON files are stored
TRANSCRIPTS_FOLDER = os.path.join(os.getcwd(), 'transcripts')

# Folder where summary JSON files are stored
SUMMARIES_FOLDER = os.path.join(os.getcwd(), 'summaries')

def call_claude_api(prompt: str) -> str:
    """
    Uses Langchain's ChatAnthropic to call Claude with the provided prompt.
    """
    claude = ChatAnthropic(
        model="claude-3-5-sonnet-20241022",
        temperature=0,
        anthropic_api_key=anthropic_api_key
    )
    response = claude.invoke(prompt)
    return response.content

def main():
    # Construct summary filename: e.g., MEDP_2024_Q4_summary.json
    summary_filename = f"{ticker}_{year}_Q{quarter}_summary.json"
    summary_filepath = os.path.join(SUMMARIES_FOLDER, summary_filename)

    # If the summary file already exists, load and output it (caching)
    os.makedirs(SUMMARIES_FOLDER, exist_ok=True)
    if os.path.exists(summary_filepath):
        with open(summary_filepath, 'r') as f:
            summary = json.load(f)
        print(json.dumps(summary, indent=2))
        return

    # Construct the transcript filename in the format: <ticker><YYYY>_<QN>_transcript.json
    transcript_filename = f"{ticker}_{year}_Q{quarter}_transcript.json"
    transcript_filepath = os.path.join(TRANSCRIPTS_FOLDER, transcript_filename)
    os.makedirs(TRANSCRIPTS_FOLDER, exist_ok=True)
    if not os.path.exists(transcript_filepath):
        raise FileNotFoundError(f"Transcript file {transcript_filename} not found in {TRANSCRIPTS_FOLDER}")

    # Load the transcript file
    with open(transcript_filepath, 'r') as f:
        transcript_data = json.load(f)

    # Combine transcript entries into one text blob
    transcript_text = "\n".join([
        f"{entry.get('speaker')}: {entry.get('text')}" for entry in transcript_data.get('transcript', [])
    ])

    # Build the full prompt for Claude
    full_prompt = (
        f"{prompt_title}\n\n{prompt_body}\n\n"
        f"{transcript_text}"
    )

    # Call the Claude API using Langchain
    summary_text = call_claude_api(full_prompt)
    
    # Truncate summary text from the beginning until it includes "Highlight Summary:"
    if "Highlight Summary:" in summary_text:
        summary_text = summary_text.split("Highlight Summary:", 1)[1]
        summary_text = "Highlight Summary:" + summary_text

    print(summary_text)

    # Attempt to parse the summary as JSON; if it fails, return the raw text
    try:
        summary = json.loads(summary_text)
    except json.JSONDecodeError:
        summary = {"summary": summary_text}

    # Save the summary by writing it to a JSON file
    with open(summary_filepath, 'w') as f:
        json.dump(summary, f, indent=2)

if __name__ == '__main__':
    main()

Highlight Summary:
- Q4 2024 Revenue: $536.6M (up 7.7% YoY)
- Full Year 2024 Revenue: $2.11B (up 11.8% YoY)
- Q4 Book-to-bill ratio: 0.99
- Ending backlog: $2.9B (up 3.2% YoY)

Summary of CEO's Comments:
- Business environment weakened somewhat in Q4
- Backlog cancellations returned to normal range in Q4
- Expects weak bookings in first half of 2025
- Projects book-to-bill ratio above 1.15 in second half of 2025
- Revenue growth for 2025 expected in low single digits

Financial Performance:
- Q4 EBITDA: $133.5M (up 39.3% YoY)
- Q4 EBITDA margin: 24.9% (vs 19.2% in prior year)
- Q4 Net Income: $117M (up 49.5% YoY)
- Full Year Net Income: $404.4M (up 43% YoY)
- Cash position: $669.4M as of December 31, 2024

Business Highlights:
- Customer concentration: Top 5 customers represent 22% of revenue
- Top 10 customers represent 29% of revenue
- Maintained high staff productivity and retention
- Planning mid to upper single-digit headcount growth in 2025
- Early stages of India operations expa